# General Functions

In [1]:
def get_input(day):
    input_day = []
    with open(f"input_day_{day}", "r") as f:
        for line in f:
            input_day.append(line[:-1])
    return input_day

# [Day 1](https://adventofcode.com/2020/day/1)
## Part A
I wrote a solution that will look for an expected sum and return the values and the multiplication. If the sum cannot be build, it will just print that no solution exists and return None.

In [2]:
input_day_1 = [int(val) for val in get_input(1)]

In [3]:
def solution_part_1(input_list, expected_sum):
    input_list_sorted = sorted(input_list)
    start = 0
    end = len(input_list_sorted)-1
    try:
        while (int_sum:=input_list_sorted[start]+input_list_sorted[end])!=expected_sum:
            if int_sum<expected_sum:
                start += 1
            elif int_sum>expected_sum:
                end -= 1
    except IndexError:
        print("No valid solution found!")
        return None
    return (input_list_sorted[start], input_list_sorted[end]),input_list_sorted[start]*input_list_sorted[end]
print(solution_part_1(input_day_1, 2020))

((267, 1753), 468051)


## Part B
I rewrote the solution so it can solve Part A and B.

In [4]:
import itertools
import math
def get_day_1_result(input_list, amount_numbers, expected_sum):
    for combination in itertools.combinations(input_list, amount_numbers):
        if sum(combination)==expected_sum:
            return combination, math.prod(combination)
print(get_day_1_result(input_day_1, 3, 2020))

((523, 551, 946), 272611658)


## Speed Comparison
Comparing the Part A Solution and Part B Solution used for A.

In [5]:
%timeit get_day_1_result(input_day_1, 2, 2020)

2.58 ms ± 170 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%timeit solution_part_1(input_day_1, 2020)

51.4 µs ± 998 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# [Day 2](https://adventofcode.com/2020/day/2)
## Part A

In [7]:
def tobbogan_password_validation(password, letter, min_amount, max_amount):
    return min_amount <= password.count(letter) <= max_amount

In [8]:
def split_line(line):
    parts = line.split(' ')
    password = parts[2]
    letter = parts[1][:-1]
    min_amount, max_amount = (parts[0].split('-'))
    return (password, letter, int(min_amount), int(max_amount))

In [9]:
result = sum([tobbogan_password_validation(*split_line(line)) for line in get_input(2)])

In [10]:
result

445

## Part B

In [11]:
def tobbogan_password_validation_2(password, letter, low_position, high_position):
    return (password[low_position-1]==letter)+(password[high_position-1]==letter)==1

In [12]:
result = sum([tobbogan_password_validation_2(*split_line(line)) for line in get_input(2)])

In [13]:
result

491

# [Day 3](https://adventofcode.com/2020/day/3)
## Part A

In [14]:
input_day_3 = get_input(3)

In [15]:
def trees_in_slope(slope, movement_to_right, movement_down):
    x_pos = 0
    trees = 0
    for line_position in range(0, len(slope), movement_down):
        line = slope[line_position]
        trees += 1 if line[x_pos%len(line)]=='#' else 0
        x_pos += movement_to_right
    return trees

In [16]:
print(trees_in_slope(input_day_3,3, 1))

286


## Part B

In [17]:
print(math.prod([trees_in_slope(input_day_3, right,down) for right, down in [(1,1),(3,1),(5,1),(7,1),(1,2)]]))

3638606400


## Personal Bonus
Find a path without trees.

In [18]:
import numpy as np
down = 1
right = 0
num_trees = trees_in_slope(input_day_3,right,down)
while num_trees > 0:
    if right>1:
        right -= 1
    else:
        down += 1
        right = down
    num_trees = min(num_trees, trees_in_slope(input_day_3,right,down))

In [19]:
print(f"No trees when going {down} down and {right} right.")

No trees when going 19 down and 7 right.


# [Day 4](https://adventofcode.com/2020/day/4)
## Part A

In [20]:
input_day_4 = get_input(4)

In [21]:
current_passport = {}
necessary_fields = {"byr", "iyr", "eyr", "hgt", "hcl", "ecl", "pid"}
valid = 0
def check_passport(passport):
    return 0 if necessary_fields.difference(passport.keys()) else 1
for line in input_day_4:
    if line == '':
        valid += check_passport(current_passport)
        current_passport = {}
    else:
        current_passport.update({entry.split(':')[0]:entry.split(':')[1] for entry in line.split(' ')})
print(valid)

256


## Part B

In [22]:
import string

def check_eye_color(ecl):
    valid_eye_colors = ["amb", "blu", "brn", "gry", "grn", "hzl", "oth"]
    return ecl in valid_eye_colors

def check_hair_color(hcl):
    hexdigits = set(string.hexdigits.lower())
    return (hcl[0]=='#' and all(c in hexdigits for c in hcl[1:]))

def check_numberstring_between(number: str, low: int, high: int):
    return low<=int(number)<=high 

def check_height(hgt):
    allowed_heights = {"cm":(150, 193), "in":(59,76)}
    measure = hgt[-2:]
    height = hgt[:-2]
    return measure in allowed_heights.keys() and check_numberstring_between(height, *allowed_heights[measure])
    
def check_birth_year(byr):
    return check_numberstring_between(byr, 1920, 2002)

def check_issue_year(iyr):
    return check_numberstring_between(iyr, 2010, 2020)

def check_expiration_year(eyr):
    return check_numberstring_between(eyr, 2020, 2030)

def check_pid(pid):
    return len(pid)==9 and pid.isnumeric()

def check_passport_extended(passport):
    checks = {
        "ecl": check_eye_color,
        "hcl": check_hair_color,
        "hgt": check_height,
        "byr": check_birth_year,
        "iyr": check_issue_year,
        "eyr": check_expiration_year,
        "pid": check_pid
    }
    try:
        check_result = all(checker(passport.get(field, "")) for field, checker in checks.items())
    except:
        check_result = False
    return check_result

In [23]:
valid = 0
current_passport = {}
for line in input_day_4:
    if line == '':
        valid += check_passport_extended(current_passport)
        current_passport = {}
    else:
        current_passport.update({entry.split(':')[0]:entry.split(':')[1] for entry in line.split(' ')})
print(valid)

198


# [Day 5](https://adventofcode.com/2020/day/5)
## Part A

In [24]:
input_day_5 = get_input(5)

In [25]:
def compute_seat_id(boarding_string):
    return int(boarding_string[:7].translate(str.maketrans("FB","01")), 2)*8+int(boarding_string[7:].translate(str.maketrans("LR","01")), 2)
seat_ids = [compute_seat_id(pass_id) for pass_id in input_day_5]
max(seat_ids)

951

## Part B

In [26]:
def find_seat(seat_list):
    seat_list = sorted(seat_list)
    for index, seat in enumerate(seat_list):
        if seat+2 == seat_list[index+1]:
            return seat+1
find_seat(seat_ids)

653

# [Day 6](https://adventofcode.com/2020/day/6)
## Part A

In [27]:
input_day_6 = get_input(6)

In [28]:
current_group = set()
answers = []
for line in input_day_6:
    if line == '':
        answers.append(len(current_group))
        current_group = set()
    else:
        current_group = current_group.union(set(line))
print(sum(answers))

6612


## Part B

In [29]:
current_group = None
answers = []
for line in input_day_6:
    if line == '':
        answers.append(len(current_group))
        current_group = None
    elif current_group is None:
        current_group = set(line)
    else:
        current_group = current_group.intersection(set(line))
print(sum(answers))

3268
